# **Simple Event Calendar Q&A**

**● Python Tasks:**
  
Pre‑processing: Load event details from a CSV (or JSON) file and format dates using Python’s datetime module.

Post‑processing: Display events in a clear, calendar-like output.

  **● Data Source:**

Personal calendar data or public event listings from local community sites.

   **● Work Outline:**

Read and index events by date, then allow user queries such as “What events are scheduled tomorrow?”

Use RAG to pick the relevant event information and output it neatly.

**● Objective:**

Design and develop a Simple Event Calendar Q&A system in Python that loads event details from a CSV or JSON file, formats dates using the datetime module, and displays events in a clear calendar-like output. The system will utilize personal calendar data or public event listings, read and index events by date, and enable user queries to retrieve relevant event information, providing a neat and organized output.

In [ ]:
pip install boltiotai

In [ ]:
from boltiotai import openai

In [ ]:
import csv

# The corrected CSV content
csv_content = """title,date,description
Republic Day,2025-01-26,Republic Day is a national holiday in India.
Independence Day,2025-08-15,Independence Day marks the end of British rule in 1947.
Gandhi Jayanti,2025-10-02,Gandhi Jayanti is celebrated in honor of Mahatma Gandhi's birthday.
Diwali,2025-11-12,Diwali is the festival of lights celebrated by Hindus.
Holi,2025-03-25,Holi is the festival of colors celebrated by Hindus.
Christmas,2025-12-25,Christmas is celebrated to commemorate the birth of Jesus Christ.
Eid al-Fitr,2025-04-22,Eid al-Fitr is celebrated by Muslims marking the end of Ramadan.
Raksha Bandhan,2025-08-10,Raksha Bandhan celebrates the bond between brothers and sisters.
Makar Sankranti,2025-01-14,Makar Sankranti is a harvest festival celebrated in various parts of India.
Janmashtami,2025-08-31,Janmashtami marks the birth of Lord Krishna.
Navratri,2025-10-01,Navratri is a festival dedicated to the worship of the Hindu deity Durga.
Good Friday,2025-04-18,Good Friday commemorates the crucifixion of Jesus Christ.
Maha Shivaratri,2025-02-27,Maha Shivaratri is a Hindu festival dedicated to Lord Shiva.
Guru Nanak Jayanti,2025-11-15,Guru Nanak Jayanti celebrates the birth of the first Sikh guru Guru Nanak.
Baisakhi,2025-04-13,Baisakhi is a harvest festival celebrated in Punjab.
Meeting with boss,2025-02-21,A meeting scheduled with the boss.
Project submission,2025-02-22,Submission deadline for the project.
Family gathering,2025-02-28,A family gathering event.
Doctor's appointment,2025-03-01,A scheduled appointment with the doctor.
"""

# Writing the corrected content to a CSV file
with open('events.csv', 'w', newline='', encoding='utf-8') as file:
    file.write(csv_content)

# To provide the file for download in Colab
from google.colab import files
files.download('events.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
import csv
import json
from datetime import datetime, timedelta
from typing import List, Dict, Union
from boltiotai import openai
from google.colab import files


class Event:
    def __init__(self, title: str, date: str, description: str):
        self.title = title
        self.date = datetime.strptime(date, "%Y-%m-%d")
        self.description = description

    def __repr__(self):
        return f"Event(title={self.title}, date={self.date.strftime('%Y-%m-%d')}, description={self.description})"

def load_events_from_csv(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8-sig') as file:
        reader = csv.DictReader(file)
        for row in reader:
            events.append(Event(row['title'], row['date'], row['description']))
    return events

def load_events_from_json(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            events.append(Event(item['title'], item['date'], item['description']))
    return events

def index_events_by_date(events: List[Event]) -> Dict[str, List[Event]]:
    indexed_events = {}
    for event in events:
        date_str = event.date.strftime('%Y-%m-%d')
        if date_str not in indexed_events:
            indexed_events[date_str] = []
        indexed_events[date_str].append(event)
    return indexed_events

def display_events(events: List[Event]):
    print("Event Name | Event Date")
    print("-" * 34)
    for event in events:
        print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

def get_events_for_date(indexed_events: Dict[str, List[Event]], date: datetime) -> List[Event]:
    date_str = date.strftime('%Y-%m-%d')
    return indexed_events.get(date_str, [])

def parse_date(date_str: str) -> Union[datetime, None]:
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        return None

def generate_response(events: List[Event], query_date: datetime) -> str:
    if not events:
        return f"No events scheduled for {query_date.strftime('%Y-%m-%d')}."

    event_details = "\n".join([f"{event.title} on {event.date.strftime('%Y-%m-%d')}" for event in events])
    prompt = f"The following events are scheduled for {query_date.strftime('%Y-%m-%d')}:\n{event_details}\n\nPlease provide a summary:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Debug: Print the full response object
    print("Full response object:")
    print(response)

    # Extract and return the content of the response
    if 'choices' in response:
        return response['choices'][0].get('message', {}).get('content', response['choices'][0].get('text', '')).strip()
    else:
        return "Unexpected response format. Please check the debug output."

def interpret_query(user_query: str) -> Union[datetime, None]:
    user_query = user_query.lower()
    if "tomorrow" in user_query:
        return datetime.now() + timedelta(days=1)
    elif "today" in user_query:
        return datetime.now()
    else:
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', user_query)
        if date_match:
            return parse_date(date_match.group(1))
        return None

def main():
    uploaded = files.upload()
    events = []
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            events = load_events_from_csv(filename)
        elif filename.endswith('.json'):
            events = load_events_from_json(filename)
        else:
            print("Unsupported file type.")
            return

    indexed_events = index_events_by_date(events)
    user_query = input("Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): ").strip().lower()

    query_date = interpret_query(user_query)
    if not query_date:
        print("Unsupported query or invalid date format. Please try again.")
        return

    events_for_query = get_events_for_date(indexed_events, query_date)

    # Display events in the format: Event Name | Event Date
    if events_for_query:
        print("Event Name | Event Date")
        print("-" * 34)
        for event in events_for_query:
            print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

    response = generate_response(events_for_query, query_date)
    print("Response:\n" + response)

if __name__ == "__main__":
    main()

Saving events.csv to events (26).csv
Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): 2025-01-26
Event Name | Event Date
----------------------------------
Republic Day | 2025-01-26
Full response object:
{'choices': [{'message': {'content': "On January 26, 2025, Republic Day is scheduled to be celebrated. Republic Day is a national holiday in India, commemorating the day the Constitution of India came into effect in 1950, marking the country's transition to a republic. The day is typically marked by parades, flag hoisting ceremonies, cultural events, and patriotic festivities."}}], 'model': 'gpt-3.5-turbo-0125', 'usage': {'completion_tokens': 73, 'prompt_tokens': 47, 'total_tokens': 120}}
Response:
On January 26, 2025, Republic Day is scheduled to be celebrated. Republic Day is a national holiday in India, commemorating the day the Constitution of India came into effect in 1950, marking the country's transition to a republic.

In [ ]:
import re
import csv
import json
from datetime import datetime, timedelta
from typing import List, Dict, Union
from boltiotai import openai
from google.colab import files


class Event:
    def __init__(self, title: str, date: str, description: str):
        self.title = title
        self.date = datetime.strptime(date, "%Y-%m-%d")
        self.description = description

    def __repr__(self):
        return f"Event(title={self.title}, date={self.date.strftime('%Y-%m-%d')}, description={self.description})"

def load_events_from_csv(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8-sig') as file:
        reader = csv.DictReader(file)
        for row in reader:
            events.append(Event(row['title'], row['date'], row['description']))
    return events

def load_events_from_json(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            events.append(Event(item['title'], item['date'], item['description']))
    return events

def index_events_by_date(events: List[Event]) -> Dict[str, List[Event]]:
    indexed_events = {}
    for event in events:
        date_str = event.date.strftime('%Y-%m-%d')
        if date_str not in indexed_events:
            indexed_events[date_str] = []
        indexed_events[date_str].append(event)
    return indexed_events

def display_events(events: List[Event]):
    print("Event Name | Event Date")
    print("-" * 34)
    for event in events:
        print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

def get_events_for_date(indexed_events: Dict[str, List[Event]], date: datetime) -> List[Event]:
    date_str = date.strftime('%Y-%m-%d')
    return indexed_events.get(date_str, [])

def parse_date(date_str: str) -> Union[datetime, None]:
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        return None

def generate_response(events: List[Event], query_date: datetime) -> str:
    if not events:
        return f"No events scheduled for {query_date.strftime('%Y-%m-%d')}."

    event_details = "\n".join([f"{event.title} on {event.date.strftime('%Y-%m-%d')}" for event in events])
    prompt = f"The following events are scheduled for {query_date.strftime('%Y-%m-%d')}:\n{event_details}\n\nPlease provide a summary:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Debug: Print the full response object
    print("Full response object:")
    print(response)

    # Extract and return the content of the response
    if 'choices' in response:
        return response['choices'][0].get('message', {}).get('content', response['choices'][0].get('text', '')).strip()
    else:
        return "Unexpected response format. Please check the debug output."

def interpret_query(user_query: str) -> Union[datetime, None]:
    user_query = user_query.lower()
    if "tomorrow" in user_query:
        return datetime.now() + timedelta(days=1)
    elif "today" in user_query:
        return datetime.now()
    else:
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', user_query)
        if date_match:
            return parse_date(date_match.group(1))
        return None

def main():
    uploaded = files.upload()
    events = []
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            events = load_events_from_csv(filename)
        elif filename.endswith('.json'):
            events = load_events_from_json(filename)
        else:
            print("Unsupported file type.")
            return

    indexed_events = index_events_by_date(events)
    user_query = input("Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): ").strip().lower()

    query_date = interpret_query(user_query)
    if not query_date:
        print("Unsupported query or invalid date format. Please try again.")
        return

    events_for_query = get_events_for_date(indexed_events, query_date)

    # Display events in the format: Event Name | Event Date
    if events_for_query:
        print("Event Name | Event Date")
        print("-" * 34)
        for event in events_for_query:
            print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

    response = generate_response(events_for_query, query_date)
    print("Response:\n" + response)

if __name__ == "__main__":
    main()

Saving events.csv to events (27).csv
Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): What events are scheduled tomorrow?
Response:
No events scheduled for 2025-02-23.


In [ ]:
import re
import csv
import json
from datetime import datetime, timedelta
from typing import List, Dict, Union
from boltiotai import openai
from google.colab import files


class Event:
    def __init__(self, title: str, date: str, description: str):
        self.title = title
        self.date = datetime.strptime(date, "%Y-%m-%d")
        self.description = description

    def __repr__(self):
        return f"Event(title={self.title}, date={self.date.strftime('%Y-%m-%d')}, description={self.description})"

def load_events_from_csv(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8-sig') as file:
        reader = csv.DictReader(file)
        for row in reader:
            events.append(Event(row['title'], row['date'], row['description']))
    return events

def load_events_from_json(file_path: str) -> List[Event]:
    events = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            events.append(Event(item['title'], item['date'], item['description']))
    return events

def index_events_by_date(events: List[Event]) -> Dict[str, List[Event]]:
    indexed_events = {}
    for event in events:
        date_str = event.date.strftime('%Y-%m-%d')
        if date_str not in indexed_events:
            indexed_events[date_str] = []
        indexed_events[date_str].append(event)
    return indexed_events

def display_events(events: List[Event]):
    print("Event Name | Event Date")
    print("-" * 34)
    for event in events:
        print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

def get_events_for_date(indexed_events: Dict[str, List[Event]], date: datetime) -> List[Event]:
    date_str = date.strftime('%Y-%m-%d')
    return indexed_events.get(date_str, [])

def parse_date(date_str: str) -> Union[datetime, None]:
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        return None

def generate_response(events: List[Event], query_date: datetime) -> str:
    if not events:
        return f"No events scheduled for {query_date.strftime('%Y-%m-%d')}."

    event_details = "\n".join([f"{event.title} on {event.date.strftime('%Y-%m-%d')}" for event in events])
    prompt = f"The following events are scheduled for {query_date.strftime('%Y-%m-%d')}:\n{event_details}\n\nPlease provide a summary:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Debug: Print the full response object
    print("Full response object:")
    print(response)

    # Extract and return the content of the response
    if 'choices' in response:
        return response['choices'][0].get('message', {}).get('content', response['choices'][0].get('text', '')).strip()
    else:
        return "Unexpected response format. Please check the debug output."

def interpret_query(user_query: str) -> Union[datetime, None]:
    user_query = user_query.lower()
    if "tomorrow" in user_query:
        return datetime.now() + timedelta(days=1)
    elif "today" in user_query:
        return datetime.now()
    else:
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', user_query)
        if date_match:
            return parse_date(date_match.group(1))
        return None

def main():
    uploaded = files.upload()
    events = []
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            events = load_events_from_csv(filename)
        elif filename.endswith('.json'):
            events = load_events_from_json(filename)
        else:
            print("Unsupported file type.")
            return

    indexed_events = index_events_by_date(events)
    user_query = input("Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): ").strip().lower()

    query_date = interpret_query(user_query)
    if not query_date:
        print("Unsupported query or invalid date format. Please try again.")
        return

    events_for_query = get_events_for_date(indexed_events, query_date)

    # Display events in the format: Event Name | Event Date
    if events_for_query:
        print("Event Name | Event Date")
        print("-" * 34)
        for event in events_for_query:
            print(f"{event.title} | {event.date.strftime('%Y-%m-%d')}")

    response = generate_response(events_for_query, query_date)
    print("Response:\n" + response)

if __name__ == "__main__":
    main()

Saving events.csv to events (28).csv
Enter your query (e.g., 'What events are scheduled tomorrow?' or 'What events are scheduled on YYYY-MM-DD?'): 2025-08-31
Event Name | Event Date
----------------------------------
Janmashtami | 2025-08-31
Full response object:
{'choices': [{'message': {'content': 'On August 31, 2025, Janmashtami will be celebrated. Janmashtami is a Hindu festival that commemorates the birth of Lord Krishna. Devotees observe fasts, sing songs, perform dances, and participate in various festivities to celebrate this auspicious occasion. It is a joyous and vibrant festival observed by millions of Hindus around the world.'}}], 'model': 'gpt-3.5-turbo-0125', 'usage': {'completion_tokens': 77, 'prompt_tokens': 49, 'total_tokens': 126}}
Response:
On August 31, 2025, Janmashtami will be celebrated. Janmashtami is a Hindu festival that commemorates the birth of Lord Krishna. Devotees observe fasts, sing songs, perform dances, and participate in various festivities to celebra